In [ ]:
# Data analysis


# libraries: 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns

In [ ]:
# data

data = pd.read_csv('data_tog.csv', index_col= 0)

In [ ]:
name_mac_relation = pd.read_csv("mac_name_relation.csv", index_col=0)
good_aps = name_mac_relation[name_mac_relation['ap_name'].isin( ["Guest-CentraleSupelec", "eduroam", 'stop&go', 'CD91', 'fabrique2024'])]["ap_mac"].to_list()
columns_to_maintain  = good_aps + data.columns[:5].to_list()
data = data[data.columns.intersection(columns_to_maintain)]


In [ ]:
columns_features = ['timestamp', 'room', 'device_id', 'door_status', 'room_part']
macColumns = [data for data in data.columns if data not in columns_features]

In [ ]:
dataV = pd.DataFrame([row for index, row in data.iterrows() if 'V' in row.values])
dataG = pd.DataFrame([row for index, row in data.iterrows() if 'G' in row.values])
dataC = pd.DataFrame([row for index, row in data.iterrows() if 'C' in row.values])
dataR = pd.DataFrame([row for index, row in data.iterrows() if 'R' in row.values])

In [ ]:
dataV.drop(columns= 'device_id')
dataG.drop(columns= 'device_id')
dataC.drop(columns= 'device_id')
dataR.drop(columns= 'device_id')

In [ ]:
mac_mapping = {mac: i+1 for i, mac in enumerate(macColumns)}

In [ ]:
from sklearn.metrics import mean_squared_error


def analysisRoom(df,door_status, room_part, room): 

    data_filtered =  df[(df['door_status'] == door_status) & (df['room_part'] == room_part) & (df['room'] == room)]
    data_filtered.rename(columns=mac_mapping, inplace=True)
    data_filtered = data_filtered.drop(columns=columns_features)
    non_null_counts = data_filtered.count()
    del(data_filtered)
    return non_null_counts

In [ ]:

# Assuming analysisRoom returns the power values for a given MAC address
non_null_counts_V = analysisRoom(dataV, 0, 4, 'LC414')
non_null_counts_G = analysisRoom(dataG, 0, 4, 'LC414')
non_null_counts_R = analysisRoom(dataR, 0, 4, 'LC414')
non_null_counts_C = analysisRoom(dataC, 0, 4, 'LC414')


# Assuming analysisRoom returns the power values for a given MAC address
dataframes = {'DataV': dataV, 'DataG': dataG, 'DataR': dataR, 'DataC': dataC}
column_names = ['DataV', 'DataG', 'DataR', 'DataC']

# Create an empty DataFrame with the MAC addresses as index
sale = 'LC426'
porte = 0 # 0, si ouverte, 1, si fermé 
part_salle = 3
combined_data = pd.DataFrame(index=analysisRoom(dataV, porte, part_salle, sale).index)

# Iterate through each dataframe and add it as a column to the combined dataframe
for column_name, dataframe in dataframes.items():
    combined_data[column_name] = analysisRoom(dataframe, porte ,part_salle, sale)

# Reset index to convert MAC addresses into a column
combined_data.reset_index(inplace=True)
combined_data.rename(columns={'index': 'MAC Address'}, inplace=True)


combined_data = combined_data.loc[(combined_data.iloc[:,2:] != 0).any(axis=1)]


In [ ]:
combined_data.drop(columns = 'MAC Address')['DataV']

In [ ]:
combined_data

In [ ]:
# Plotting the histogram with transposed data using a bar plot
plt.bar(combined_data['MAC Address'], combined_data['DataV'])  # Plotting the bar plot
#plt.bar(combined_data['MAC Address'], combined_data['DataC'],color ='black')  # Plotting the bar plot
plt.xlabel('MAC Address')  # Assuming you want to label the x-axis with index values
plt.ylabel('# of measures')  # Assuming the y-axis represents the frequency of values
plt.title('Salle {salle} - {porte1} - {part_salle1}'.format(salle = sale, porte1 = porte, part_salle1 = part_salle))  # Title of the plot

plt.grid(True)  # Show gridlines
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Create a figure and axis objects for subplots
fig, axs = plt.subplots(1, 3, figsize=(15, 5))

# Plotting the first set of data
axs[0].bar(combined_data['MAC Address'], combined_data['DataV'], color=(0,1,1,0.8))
axs[0].bar(combined_data['MAC Address'], combined_data['DataG'], color=(1,1,0,0.8))
axs[0].set_xlabel('MAC Address')
axs[0].set_ylabel('# of measures')
axs[0].set_title('Salle {salle} - {porte1} - {part_salle1}'.format(salle = sale, porte1 = porte, part_salle1 = part_salle))
axs[0].legend(['Data_Vitor', 'Data_Guilherme'])
axs[0].grid(True)

# Plotting the second set of data
axs[1].bar(combined_data['MAC Address'], combined_data['DataV'], color=(0,1,1,0.8))
axs[1].bar(combined_data['MAC Address'], combined_data['DataC'], color=(1,1,0,0.8))
axs[1].set_xlabel('MAC Address')
axs[1].set_ylabel('# of measures')
axs[1].set_title('Salle {salle} - {porte1} - {part_salle1}'.format(salle = sale, porte1 = porte, part_salle1 = part_salle))
axs[1].legend(['Data_Vitor', 'Data_Corentin'])
axs[1].grid(True)

# Plotting the third set of data
axs[2].bar(combined_data['MAC Address'], combined_data['DataV'], color=(0,1,1,0.8))
axs[2].bar(combined_data['MAC Address'], combined_data['DataR'], color=(1,1,0,0.8))
axs[2].set_xlabel('MAC Address')
axs[2].set_ylabel('# of measures')
axs[2].set_title('Salle {salle} - {porte1} - {part_salle1}'.format(salle = sale, porte1 = porte, part_salle1 = part_salle))
axs[2].legend(['Data_Vitor', 'Data_Ricardo'])
axs[2].grid(True)

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:


def analysisRoom_MSE(df, df_ref, door_status, room_part, room):
    data_filtered = df[(df['door_status'] == door_status) & (df['room_part'] == room_part) & (df['room'] == room)].copy()
    data_filtered_ref = df_ref[(df_ref['door_status'] == door_status) & (df_ref['room_part'] == room_part) & (df_ref['room'] == room)].copy()
    data_filtered.rename(columns=mac_mapping, inplace=True)
    data_filtered_ref.rename(columns=mac_mapping, inplace=True)
    data_filtered = data_filtered.drop(columns=columns_features)
    data_filtered_ref = data_filtered_ref.drop(columns=columns_features)

    # Convertir NaN a 0
    data_filtered = np.nan_to_num(data_filtered)
    data_filtered_ref = np.nan_to_num(data_filtered_ref)

    MSE = mean_squared_error(data_filtered, data_filtered_ref)
    return MSE


In [176]:
MSE_results = pd.DataFrame(index=analysisRoom(dataV, porte, part_salle, sale).index)


for column_name, dataframe in dataframes.items():
    print(column_name)
    if column_name!= 'DataV':
        MSE_results[column_name] = analysisRoom_MSE(dataframe, dataV, porte, part_salle, sale)

MSE_results


DataV
DataG
DataR
DataC


C:\Users\vopsf\AppData\Local\Temp\ipykernel_9512\1403962888.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered.rename(columns=mac_mapping, inplace=True)


,DataG,DataR,DataC
1,486.606797,418.589731,334.599853
2,486.606797,418.589731,334.599853
3,486.606797,418.589731,334.599853
4,486.606797,418.589731,334.599853
5,486.606797,418.589731,334.599853
...,...,...,...
405,486.606797,418.589731,334.599853
406,486.606797,418.589731,334.599853
407,486.606797,418.589731,334.599853
408,486.606797,418.589731,334.599853


In [ ]:
dataframes.keys()